# Deteccion de texto usando Amazon Rekognition

In [ ]:
!conda upgrade -y boto3

In [ ]:
import boto3
print(boto3.__version__)

import botocore
print(botocore.__version__)

In [ ]:
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
bucketName = "{REPLACE_YOUR_BUCKET}"
imageName = "media/coffee.jpg"

In [ ]:
s3.upload_file(
    Filename="resources/coffee.jpg",
    Bucket=bucketName,
    Key=imageName,
)

In [ ]:
detectTextResponse = rekognition.detect_text(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
      },
      Filters={
        'WordFilter': {
            'MinConfidence': 90
        }
      }
)

In [ ]:
display(detectTextResponse)

In [ ]:
import string
unsafeWords = ["crap", "darn", "damm"]
for textDetection in detectTextResponse["TextDetections"]:
    # strip punctuation before checking match
    text = textDetection["DetectedText"].translate(str.maketrans('', '', string.punctuation))
    if(textDetection["Type"] == "WORD" and text in unsafeWords):
        print("Detected unsafe word: {}".format(textDetection["DetectedText"]))

In [ ]:
def drawBoundingBoxes (sourceImage, boxes):
    colors = ((255,255,255),(255,255,255),(76,182,252),(52,194,123))
    
    imageLocation = 'resources/'+os.path.basename(sourceImage)

    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line= 3
    for box in boxes:
        x1 = int(box[1]['Left'] * width)
        y1 = int(box[1]['Top'] * height)
        x2 = int(box[1]['Left'] * width + box[1]['Width'] * width)
        y2 = int(box[1]['Top'] * height + box[1]['Height']  * height)
        
        draw.text((x1,y1),box[0],colors[col])
        for l in range(line):
            draw.rectangle((x1-l,y1-l,x2+l,y2+l),outline=colors[col])
        col = (col+1)%maxcol
    
    imageFormat = "PNG"
    ext = sourceImage.lower()
    if(ext.endswith('jpg') or ext.endswith('jpeg')):
        imageFormat = 'JPEG'

    bbImage.save(imageLocation,format=imageFormat)

    display(bbImage)

In [ ]:
boxes = []
textDetections = detectTextResponse['TextDetections']
for textDetection in textDetections:
        boxes.append ((textDetection['Type'], textDetection["Geometry"]['BoundingBox']))
    
drawBoundingBoxes(imageName, boxes)